In [1]:
import pandas as pd
import numpy as np
import pickle
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

2024-10-14 17:11:32.397741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 17:11:32.600667: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-14 17:11:34.113214: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2024-10-14 17:11:34.113322: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2024-10-14 17:11:34.120380: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd

In [3]:
# windows
test_img = pd.read_pickle(r'./mutation_images/test_imgs_v1.pkl')
train_img = pd.read_pickle(r'./mutation_images/train_imgs_v1.pkl')
# wsl
# test_img = pd.read_pickle(r'/mnt/c/Github/competition/gene2024/mutation_images/test_imgs_v1.pkl')
# train_img = pd.read_pickle(r'/mnt/c/Github/competition/gene2024/mutation_images/train_imgs_v1.pkl')

In [4]:
# img_dict = pd.read_pickle(r'.\mutation_images\subclass_img_dict_v1.pkl')

In [5]:
np_train = np.array(train_img)
np_train.shape

(6201, 195, 195)

In [6]:
# 두번째부터 여기부터 실행하기 train,test 는 전처리가 끝났음, 바로 모델에 학습시키면 됨
import pandas as pd
train = pd.read_csv('./open/train_preprocessed.csv')
train = train.iloc[:,1:]
test = pd.read_csv('./open/test_preprocessed.csv')
test = test.iloc[:,1:]

In [7]:
from sklearn.preprocessing import LabelEncoder
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

In [8]:
# 1. 데이터 준비
X = train.drop(columns=['ID','SUBCLASS'])
y = train['SUBCLASS']

In [9]:
# 이미지 데이터 전처리
x_train_val,x_test,y_train_val,y_test = train_test_split(np_train,y,random_state=42,test_size=0.2,stratify=y)
x_train,x_val,y_train,y_val = train_test_split(x_train_val,y_train_val,random_state=42,test_size=0.2,stratify=y_train_val)

In [10]:
x_train.shape, x_val.shape, x_test.shape

((3968, 195, 195), (992, 195, 195), (1241, 195, 195))

In [11]:
# import numpy as np
def reshape4d(array_in):
    # 데이터프레임의 행 개수 구하기
    # (3968, 195, 195,1) 배열로 변환
    num_rows = array_in.shape[0]
    return np.array(array_in.reshape(num_rows, 195, 195, 1))

# print("변환된 배열의 형태:", reshaped_array.shape)
np_4d_train = reshape4d(x_train)
np_4d_val = reshape4d(x_val)
np_4d_test = reshape4d(x_test)
np_4d_train.shape

(3968, 195, 195, 1)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, AveragePooling2D, AveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

tf.random.set_seed(42)
batch_size = 128
epochs = 100
num_classes = len(y.unique())

# 모델 정의
model = Sequential([
    # 입력 데이터 형태는 (65, 65, 1)
    Conv2D(64, kernel_size=5, strides=1, input_shape=(195, 195, 1)),
    Activation('relu'),
    Conv2D(64, kernel_size=5, strides=1),
    Activation('relu'),
    MaxPooling2D(pool_size=2),
    # AveragePooling2D(pool_size=2),
    
    Conv2D(128, kernel_size=3, strides=1),
    Activation('relu'),
    Conv2D(128, kernel_size=5, strides=1),
    Activation('relu'),
    MaxPooling2D(pool_size=2),
    # AveragePooling2D(pool_size=2),
    
    # Conv2D(256, kernel_size=5, strides=1),
    # Activation('relu'),
    # Conv2D(256, kernel_size=5, strides=1),
    # Activation('relu'),
    # Conv2D(256, kernel_size=5, strides=1),
    # Activation('relu'),
    # MaxPooling2D(pool_size=2),
    # # AveragePooling2D(pool_size=2),

    # Conv2D(256, kernel_size=5, strides=1),
    # Activation('relu'),
    # Conv2D(256, kernel_size=5, strides=1),
    # Activation('relu'),
    # Conv2D(256, kernel_size=5, strides=1),
    # Activation('relu'),
    # MaxPooling2D(pool_size=2),

    # 분류 레이어
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax'),
])

model.summary()

# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy']) # sparse_categorical_crossentropy

# 콜백 정의
callbacks = [EarlyStopping(monitor='val_accuracy', patience=5, verbose=0)]

# 모델 학습
history = model.fit(np_4d_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=(np_4d_val, y_val))

# 모델 평가
scores = model.evaluate(np_4d_test, y_test, verbose=1)

# 모델 저장
model.save('conv_model001.keras')

# 모델 불러오기
loaded_model = load_model('conv_model001.keras')

# 불러온 모델 평가 (확인용)
loaded_scores = loaded_model.evaluate(x_test, y_test, verbose=1)
print(loaded_scores)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 191, 191, 64)      1664      
                                                                 
 activation_4 (Activation)   (None, 191, 191, 64)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 187, 187, 64)      102464    
                                                                 
 activation_5 (Activation)   (None, 187, 187, 64)      0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 93, 93, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 91, 91, 128)       73856     
                                                      

2024-10-14 17:12:34.679645: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - ETA: 0s - loss: 2.7941 - accuracy: 0.2369 

2024-10-14 17:19:15.722939: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-10-14 17:19:15.752754: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-14 17:19:15.752948: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14506 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2024-10-14 17:19:15.770316: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-14 17:19:15.770379: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

31/31 [==============================] - 417s 13s/step - loss: 2.7941 - accuracy: 0.2369 - val_loss: 2.2248 - val_accuracy: 0.3317
Epoch 2/100
31/31 [==============================] - 401s 13s/step - loss: 1.7824 - accuracy: 0.4612 - val_loss: 2.1714 - val_accuracy: 0.3327
Epoch 3/100
31/31 [==============================] - 403s 13s/step - loss: 1.0349 - accuracy: 0.6830 - val_loss: 2.6714 - val_accuracy: 0.2974
Epoch 4/100
31/31 [==============================] - 404s 13s/step - loss: 0.7151 - accuracy: 0.7752 - val_loss: 2.8074 - val_accuracy: 0.3034
Epoch 5/100
31/31 [==============================] - 404s 13s/step - loss: 0.5554 - accuracy: 0.8259 - val_loss: 3.9379 - val_accuracy: 0.2853
Epoch 6/100
31/31 [==============================] - 403s 13s/step - loss: 0.4245 - accuracy: 0.8546 - val_loss: 5.8778 - val_accuracy: 0.2903
Epoch 7/100
39/39 [==============================] - 23s 510ms/step - loss: 3.4417 - accuracy: 0.2917


2024-10-14 18:00:10.023861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-10-14 18:00:10.054215: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-14 18:00:10.054485: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14506 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2024-10-14 18:00:10.056909: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-14 18:00:10.057003: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

39/39 [==============================] - 17s 427ms/step - loss: 3.4417 - accuracy: 0.2917
[3.4417216777801514, 0.29170024394989014]
